In [1]:
import os
import re
import yaml
import pickle

import ipyplot
import pandas as pd

import tensorflow as tf

# Data

In [2]:
cd ../../src

/Users/cock/kDrive/PhD/Projects/Labs/beerslaw-lab/src


In [3]:
experiment_path = '../experiments/incremental/fourth round/stateaction_secondslstm/binconcepts/lstm/raw_full/2021_12_27_0/'

# Gridsearch folds
gs_file = os.listdir(experiment_path + '/gridsearch results/')[0]
with open(experiment_path + '/gridsearch results/' + gs_file, 'rb') as fp:
    gs = pickle.load(fp)
    gs = gs.get_results()
    folds = gs[0]['fold_index']

# oversampling indices
xval_path = os.listdir(experiment_path + '/results/')[0]
with open(experiment_path + '/results/' + xval_path, 'rb') as fp:
    xval_object = pickle.load(fp)
    oversampled_indices = xval_object[0]['oversample_indices']
    test_indices = xval_object[0]['test_indices']
    
# id dictionary
with open(experiment_path + 'config.yaml', 'rb') as fp:
    config = pickle.load(fp)
    id_dictionary = config['id_dictionary']

In [4]:
cd ../notebooks/8 - timelines/

/Users/cock/kDrive/PhD/Projects/Labs/beerslaw-lab/notebooks/8 - timelines


In [5]:
timelines_path = '../../reports/timelines/colour timelines/'
addresses = os.listdir(timelines_path)
addresses = [ad for ad in addresses if 'png' in ad]
local_addresses = [(timelines_path + addr) for addr in addresses]

# Functions

In [6]:
def get_id_folds(folds:dict, oversampled_indices:list, id_dictionary:dict):
    learner_id_fold = {
        'train': {idd:[] for idd in id_dictionary['index']},
        'validation': {}
    }
    for fold in folds:
        train = folds[fold]['train']
        validation = folds[fold]['validation']
        
        train = [oversampled_indices[idx] for idx in train]
        validation = [oversampled_indices[idx] for idx in validation]
        
        train = [id_dictionary['sequences'][idx]['learner_id'] for idx in train]
        validation = [id_dictionary['sequences'][idx]['learner_id'] for idx in validation]
        
        for lid in train:
            learner_id_fold['train'][lid].append(fold)
        for lid in validation:
            learner_id_fold['validation'][lid] = fold
    return learner_id_fold


In [7]:
def images_labels(addresses:list, local_addresses:list, id_folds:dict):
    permutation = re.compile('^p([0-9]+)_')
    learner_id = re.compile('_l([A-z0-9]+)_t')
    task = re.compile('_t([0-9]+).')
    
    not_train = 0
    missing_perm = 0
    
    learners = []
    tasks = []
    folds = []
    ads = []
    for i, address in enumerate(addresses):
        try:
            p = permutation.findall(address)[0]
            l = learner_id.findall(address)[0]
            t = task.findall(address)[0]
            f = id_folds['validation'][l]

            ads.append(local_addresses[i])
            learners.append(l)
            tasks.append(t)
            folds.append(f)
        except IndexError:
            missing_perm += 1
        except KeyError:
            not_train += 1
            
    print('There are {} missing perms and {} timelines who were too short'.format(missing_perm, not_train))
        
    return ads, learners, tasks, folds

In [8]:
def plot_for_task(addresses:list, folds:list, tasks:list, task:int):
    indices = [i for i in range(len(tasks)) if tasks[i] == str(task)]
    ad = [addresses[idx] for idx in indices]
    fs = [folds[idx] for idx in indices]
    ipyplot.plot_class_tabs(ad, fs, max_imgs_per_tab=10, img_width=700)
    

# images

In [9]:
learner_id_fold = get_id_folds(folds, oversampled_indices, id_dictionary)
ads, learners, tasks, fs = images_labels(addresses, local_addresses, learner_id_fold)
plot_for_task(ads, fs, tasks, 2)

There are 247 missing perms and 795 timelines who were too short
